In [1]:
!python ./MaskGIT-pytorch/training_transformer.py --batch-size=10 --num-codebook-vectors=16384

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.
8.19kB [00:00, 362kB/s]                                                         
loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.
  0%|                    | 1/390 [00:06<20:25,  3.15s/it, Transformer_Loss=9.48]^C


In [1]:
!python ./MaskGIT-pytorch/training_transformer.py --batch-size=10 --num-codebook-vectors=16384 --epochs=40

Traceback (most recent call last):
  File "./MaskGIT-pytorch/training_transformer.py", line 20, in <module>
    from transformer import VQGANTransformer
  File "/home/ygong2832/W4995DL-Final/MaskGIT-pytorch/transformer.py", line 23, in <module>
    from models.vqgan import VQModel
  File "/home/ygong2832/taming-transformers/taming/models/vqgan.py", line 8, in <module>
    from taming.modules.vqvae.quantize import VectorQuantizer2 as VectorQuantizer
  File "/home/ygong2832/taming-transformers/taming/modules/vqvae/quantize.py", line 6, in <module>
    from einops import rearrange
ModuleNotFoundError: No module named 'einops'


In [19]:
!nvidia-smi

Tue Mar 29 04:53:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    67W / 149W |      0MiB / 11441MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!python ./MaskGIT-pytorch/training_transformer.py --batch-size=10 --num-codebook-vectors=16384

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.
loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.
  3%|▌                     | 10/390 [00:43<27:34,  4.35s/it, Transformer_Loss=7]
Traceback (most recent call last):
  File "./MaskGIT-pytorch/training_transformer.py", line 99, in <module>
    train_transformer = TrainTransformer(args)
  File "./MaskGIT-pytorch/training_transformer.py", line 21, in __init__
    self.train(args)
  File "./MaskGIT-pytorch/training_transformer.py", line 33, in train
    self.optim.step()
  File "/opt/conda/lib/python3.7/site-packages/torch/optim/optimizer.py", line 88, in wrapper
    return func(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/torch/autograd/grad_mode.py", line 28, in decorate_context
    return func(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/torch/optim/adamw.py", line 148, in step
    eps=group['eps'])
  File "/opt/con

In [2]:
ls ./taming-transformers/logs/vqgan_imagenet_f16_16384/configs/

model.yaml


In [26]:
from omegaconf import OmegaConf
config_path = r"./taming-transformers/logs/vqgan_imagenet_f16_16384/configs/model.yaml"
config = OmegaConf.load(config_path)
config.model.params['n_embed']

16384

In [ ]:
import os
import numpy as np
from tqdm import tqdm
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import utils as vutils
from transformer import VQGANTransformer
from utils import load_data, plot_images
import sys
sys.path.insert(0, '/home/ygong2832/taming-transformers')
sys.path.insert(0, '/home/ygong2832/taming-transformers/taming')
# import model
model = VQGANTransformer(args).to(device=args.device)
model.load_state_dict(torch.load('path'))
model.eval()
# test
with tqdm(range(len(test_dataset))) as pbar:
    for i, imgs in zip(pbar, test_dataset):
        real = imgs.to(device=args.device)
        vutils.save_image(real, os.path.join("real", f"{i}.jpg"))
        logits, target = model(real)
        vutils.save_image(real, os.path.join("fake", f"{i}.jpg"))